# Fine-tuning pre-trained model

In [1]:
import models, dataUtils, trainUtils
from importlib import reload
import torch, torch.nn as nn, torch.optim as optim
from torch.optim import lr_scheduler
reload(models); reload(dataUtils); reload(trainUtils)
TRAIN_DATA_PATH = './data/demoSample/'
PRETRAINED_MODEL_PATH = './modelWts/modelState_Apr06-12:31_Acc_100.00%.pth'

# Split train and val in a 70:30 ratio
dataUtils.splitTrainVal(TRAIN_DATA_PATH)

# Loading data
dataLoaders = {
    'train':dataUtils.loadPdfData(dataPath=TRAIN_DATA_PATH + 'train/', batchSize=4, 
                                  rotationAngle=1, shear=1, scalePerc=0.05, translatePerc=0.02, 
                                  brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1), 
    'val':dataUtils.loadPdfData(dataPath=TRAIN_DATA_PATH + 'val/', batchSize=4)}

# Define model architecture on GPU
model = models.resnet18(inChannels=4, num_classes=2).cuda()

# Instantiate loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer  = optim.Adam(model.parameters(), lr=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.1)

# Load pretrained model and optimizer states wts 
modelWts = torch.load(PRETRAINED_MODEL_PATH)
model.load_state_dict(modelWts['model'].state_dict())
optimizer.load_state_dict(modelWts['optimizer'].state_dict())

Spliting data into training and cross-validation in a 70:30 ratio...
Data Loaded - # Samples: 316 | Image Shape: torch.Size([4, 4, 1100, 850]) | Label Shape: torch.Size([4, 1])
Data Loaded - # Samples: 136 | Image Shape: torch.Size([4, 4, 1100, 850]) | Label Shape: torch.Size([4, 1])
Model Instantiated. # of parameters: 3.95e+07


In [2]:
# Fine Tune model
model = trainUtils.trainModel(model=model, optimizer=optimizer, criterion=criterion, scheduler=scheduler, dataLoaders=dataLoaders, numEpochs=10)

-->|15:00:42| Begin Training ...

-->|15:01:05| Epoch 0/9 train - Loss:0.3953 | Accuracy:84.49% | LR:1.0e-04
-->|15:01:15| Epoch 0/9 val - Loss:0.9004 | Accuracy:50.00% | LR:1.0e-04

-->|15:01:39| Epoch 1/9 train - Loss:0.3417 | Accuracy:85.44% | LR:1.0e-04
-->|15:01:48| Epoch 1/9 val - Loss:0.0199 | Accuracy:100.00% | LR:1.0e-04

-->|15:02:13| Epoch 2/9 train - Loss:0.2580 | Accuracy:88.92% | LR:1.0e-04
-->|15:02:23| Epoch 2/9 val - Loss:0.0566 | Accuracy:100.00% | LR:1.0e-04

-->|15:02:48| Epoch 3/9 train - Loss:0.1708 | Accuracy:93.67% | LR:1.0e-04
-->|15:02:57| Epoch 3/9 val - Loss:0.0048 | Accuracy:100.00% | LR:1.0e-04

-->|15:03:22| Epoch 4/9 train - Loss:0.0976 | Accuracy:97.47% | LR:1.0e-04
-->|15:03:32| Epoch 4/9 val - Loss:0.0020 | Accuracy:100.00% | LR:1.0e-04

-->|15:03:57| Epoch 5/9 train - Loss:0.0488 | Accuracy:99.37% | LR:1.0e-04
-->|15:04:06| Epoch 5/9 val - Loss:0.0062 | Accuracy:100.00% | LR:1.0e-04

-->|15:04:31| Epoch 6/9 train - Loss:0.0269 | Accuracy:99.68% | LR:

# Testing on new data 

In [3]:
# Ensure data is split as change and no change inorder to calculate accuracy
reload(trainUtils)
TEST_DATA_PATH = './data/XFACT_SNAP_SAMPLE_GENERATED/'

# Load test data
dataLoader = dataUtils.loadPdfData(dataPath=TEST_DATA_PATH, batchSize=2)

#test
trainUtils.testModel(model=model, dataLoader=dataLoader)

Data Loaded - # Samples: 5658 | Image Shape: torch.Size([2, 4, 1100, 850]) | Label Shape: torch.Size([2, 1])
-->|15:06:25| Begin Testing ...

Testing complete in 5m 54s
Acc: 100.00%
Confidence 0.50 | No_Change Precision: 100.00% | No_Change Recall: 100.00%
Confidence 0.80 | No_Change Precision: 100.00% | No_Change Recall: 100.00%
Confidence 0.85 | No_Change Precision: 100.00% | No_Change Recall: 100.00%
Precision- % of correctly predicted no_change/total predicted no_change | Recall- % of correctly predicted no_change/total actual no_change
